In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy.stats import zscore

csv_path = "./data/Landslide2.csv"
device = "101"
save_path = f"./data/dev{device}_resample1T.csv"
#save_path = f"data/devAll2_prepared.csv"

df = pd.read_csv(csv_path)

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

#df = df[~df['timestamp'].str.contains(r'\(ICT\)', na=False)]
#df['timestamp'] = df['timestamp'].str.replace(r'\(ICT\)', '', regex=True).str.strip()

def clean_and_format_timestamp(val):
    if pd.isnull(val):
        return None
    val = str(val).replace('(ICT)', '').strip()
    try:
        dt = pd.to_datetime(val)
        return dt.isoformat()
    except Exception:
        return None  

df['timestamp'] = df['timestamp'].apply(clean_and_format_timestamp)

df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])  # ต้องมี timestamp และ geo


df = df.drop(columns=['unixt', 'gateway time', 'old time', 'risk_level', 'confidence'], errors='ignore')
df = df[df['devID'] != 218499049.0]
df = df[(df['lat'] > 1.0) & (df['lat'] < 8000.0)]
df = df[df['lng'] > 15.0]
df = df[df['soil'] != 0.0]
df = df[df['rain'] != -1.0]

df = df[df['devID'] == int(device)]

C:\Users\ahmad\AppData\Local\Temp\ipykernel_14516\656270998.py:12: DtypeWarning: Columns (9,10,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


In [35]:
#start_date = "2025-05-02"
#end_date = "2025-05-04"
#
#df = df[(df['timestamp'] >= start_date) & (df['timestamp'] <= end_date)]
#print(df['timestamp'].min())
#print(df['timestamp'].max())

In [36]:
df.describe()

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng
count,149607,149607.0,149607.000000,149607.000000,149607.000000,149607.000000,149607.000000,149607.000000,149607.000000
mean,2025-06-04 12:12:35.064885248,101.0,48.239375,0.220004,29.755443,75.709608,11.315496,8.638010,99.898969
min,2025-05-02 11:42:06.635276,101.0,27.299999,0.000000,24.279013,46.977417,0.000000,8.637940,99.166664
25%,2025-05-29 17:45:20.274356992,101.0,44.400002,0.000000,27.405968,70.947433,5.311314,8.638001,99.898964
50%,2025-06-10 20:02:30.664009984,101.0,49.000000,0.000000,28.474098,79.242622,11.374877,8.638014,99.898972
75%,2025-06-17 11:49:32.409999872,101.0,55.400002,0.000000,32.319370,80.932556,15.343357,8.638022,99.898987
max,2025-06-29 20:26:13.090000,101.0,66.599998,41.630600,38.586636,87.270775,32.062466,8.638072,99.899498
std,NaN,0.0,9.134768,1.692000,3.059886,7.981286,6.957680,0.000019,0.001893


In [37]:
# เรียงลำดับตาม devID และ timestamp
df = df.sort_values('timestamp').reset_index(drop=True)
#df = df.sort_values(['devID', 'timestamp']).reset_index(drop=True)

# ตั้ง timestamp เป็น index
df = df.set_index('timestamp')
df

,devID,soil,rain,temp,humi,geo,lat,lng
timestamp,,,,,,,,
2025-05-02 11:42:06.635276,101.0,28.799999,0.0,34.290073,62.429081,14.532875,8.637999,99.899010
2025-05-02 11:42:18.056572,101.0,28.799999,0.0,34.266041,64.082787,14.673310,8.637998,99.899010
2025-05-02 11:42:28.667259,101.0,28.900000,0.0,34.250019,62.839169,15.995875,8.637998,99.899010
2025-05-02 11:42:40.117564,101.0,28.799999,0.0,34.185932,62.661785,19.812029,8.637998,99.899010
2025-05-02 11:42:50.732852,101.0,28.900000,0.0,34.175251,63.481953,8.099698,8.637998,99.899010
...,...,...,...,...,...,...,...,...
2025-06-29 20:20:49.190000,101.0,27.900000,0.0,29.483482,80.398491,8.771909,8.638011,99.898972
2025-06-29 20:21:12.230000,101.0,27.900000,0.0,29.528877,80.196312,8.352998,8.638011,99.898972
2025-06-29 20:25:27.980000,101.0,27.900000,0.0,29.101625,81.113754,2.490953,8.638010,99.898972


In [38]:
window = '1T'  # 10 นาที
features = ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']


df_rolling = df[features].resample(window).agg(['mean', 'std', 'min', 'max'])
df_rolling.columns = ['_'.join(col) for col in df_rolling.columns]
df_rolling['devID'] = int(device)
df_rolling = df_rolling.dropna()

df_rolling = df_rolling.reset_index()
df_rolling = df_rolling[['timestamp', 'devID', 'soil_mean', 'rain_mean', 'temp_mean', 'humi_mean', 'geo_mean']]
df_rolling

C:\Users\ahmad\AppData\Local\Temp\ipykernel_14516\4274154225.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_rolling = df[features].resample(window).agg(['mean', 'std', 'min', 'max'])


,timestamp,devID,soil_mean,rain_mean,temp_mean,humi_mean,geo_mean
0,2025-05-02 11:42:00,101,28.839999,0.0,34.233463,63.098955,14.622758
1,2025-05-02 11:43:00,101,28.816666,0.0,34.047519,63.319192,10.538904
2,2025-05-02 11:44:00,101,28.799999,0.0,34.081788,63.662011,9.419168
3,2025-05-02 11:45:00,101,28.799999,0.0,34.297550,63.891277,9.548765
4,2025-05-02 11:46:00,101,28.799999,0.0,34.542156,63.798203,10.704843
...,...,...,...,...,...,...,...
30910,2025-06-29 20:14:00,101,27.900000,0.0,30.045587,77.267529,9.788215
30911,2025-06-29 20:15:00,101,27.900000,0.0,29.556468,79.709292,7.476175
30912,2025-06-29 20:16:00,101,27.900000,0.0,29.560031,79.591670,9.711239
30913,2025-06-29 20:20:00,101,27.900000,0.0,29.868010,78.443428,8.421140


In [39]:
df_rolling.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30915 entries, 0 to 30914
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  30915 non-null  datetime64[ns]
 1   devID      30915 non-null  int64         
 2   soil_mean  30915 non-null  float64       
 3   rain_mean  30915 non-null  float64       
 4   temp_mean  30915 non-null  float64       
 5   humi_mean  30915 non-null  float64       
 6   geo_mean   30915 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 1.7 MB


In [40]:
nan_counts = df_rolling.isna().sum()

nan_columns = nan_counts[nan_counts > 0]

print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
Series([], dtype: int64)


In [41]:

df_rolling.to_csv(f"{save_path}", index=False)

print("✅ บันทึก เรียบร้อยแล้ว", save_path)

✅ บันทึก เรียบร้อยแล้ว ./data/dev101_resample1T.csv
